In [136]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

In [137]:
# df =  pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2019')
#df =  pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2020')
#df =  pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2021')
df =  pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2022')

In [138]:
df.head(1)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Кабардино-Балкарская Республика,Unnamed: 5,Unnamed: 6,Unnamed: 7,Калиниградская область,Unnamed: 9,...,Unnamed: 54,Unnamed: 55,г. Севастополь,Unnamed: 57,Unnamed: 58,Unnamed: 59,Смоленская область,Unnamed: 61,Unnamed: 62,Unnamed: 63
0,№ показателя,Наименование индикатора/показателя,№ строки,Единица измерения,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 202...,Фактическое значение нарастающим итогом за 202...,Фактическое значение показателя с нарастающим ...


In [139]:
# получаем список регионов
lst_regions = [region for region in df.columns if 'Unnamed' not in region]

lst_regions


['Кабардино-Балкарская Республика',
 'Калиниградская область',
 'Костромская область',
 'Курганская область',
 'Ленинградская область',
 'Липецкая область',
 'Мурманская область',
 'Орловская область',
 'Псковская область',
 'Республикаа Адыгея',
 'Республика Карелия',
 'Республика Марий Эл',
 'Республика Мордовия',
 'г. Севастополь',
 'Смоленская область']

In [140]:
# te_df = pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2019',skiprows=1)
#te_df = pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2020',skiprows=1)
#te_df = pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2021',skiprows=1)
te_df = pd.read_excel('data/СВОД (7).xlsx',sheet_name='Свод_2022',skiprows=1)

In [141]:
te_df.drop(0,axis=0,inplace=True)

In [142]:
te_df.drop(columns=['№ показателя','Единица измерения'],axis=1,inplace=True)

In [143]:
te_df.head(2)

,Наименование индикатора/показателя,№ строки,Планируемое значение в 2022 году,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!),"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП,Планируемое значение в 2022 году.1,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).1,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).1",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.1,...,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).12",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.12,Планируемое значение в 2022 году.13,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).13,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).13",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.13,Планируемое значение в 2022 году.14,Фактическое значение по итогам IV квартала 2022 г. (указывается значение только за 4-й квартал!).14,"Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов).14",Фактическое значение показателя с нарастающим итогом от года создания ЦОПП.14
1,Создана цифровая платформа ЦОПП в сети Интернет,1,да,Да,Да,Да,да,да,да,да,...,да,да,да,да,да,да,да,да,да,да
2,"Численность граждан Российской Федерации, охва...",2,0,1531,4002,4002,4000,3213,7857,7857,...,4265,4265,4250,3123,6033,6033,4000,4465,5160,5160


In [144]:
te_df.drop(columns=[column for column in te_df.columns if 'только за 4-й квартал!' in column],inplace=True)

In [145]:
te_df.drop(columns=['№ строки'],inplace=True)

In [146]:
lin_df = te_df.transpose()

In [147]:
# Утраиваем
lst_triple = [None]
for region in lst_regions:
    for  i in range(3):
        lst_triple.append(region)


In [148]:
# Вставляем
lin_df.insert(0,'Наименование субъекта РФ',lst_triple)

In [149]:
# Создаем объект openpyxl
wb = openpyxl.Workbook()


# получаем датафрейм
temp = lin_df[2].to_frame()

temp = temp.transpose()

temp.head(1)

base_df = temp.iloc[0,:4].to_frame().transpose()

base_df

for i in range(4,temp.shape[1],3):
    t_df = temp.iloc[0,i:i+3].to_frame().transpose()
    t_df.columns = ['Планируемое значение в 2022 году','Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)','Фактическое значение показателя с нарастающим итогом от года создания ЦОПП']
    t_df.insert(0,'Наименование индикатора/показателя',temp.iloc[0,0])
    t_df.to_excel('dt.xlsx')
    
    base_df = pd.concat([base_df,t_df],ignore_index=True,axis=0)
    
    

In [150]:
# Получаем список без первых 2 колонок
lst_col = list(lin_df.columns)[1:]

In [151]:
for idx,column in enumerate(lst_col):
    temp = lin_df[column].to_frame().transpose()
    
    # Создаем базовый датафрейм
    base_df = temp.iloc[0,:4].to_frame().transpose()
    for i in range(4,temp.shape[1],3):
        t_df = temp.iloc[0,i:i+3].to_frame().transpose()
        t_df.columns = ['Планируемое значение в 2022 году','Фактическое значение нарастающим итогом за 2022 г. (сумма I, II, III, IV кварталов)','Фактическое значение показателя с нарастающим итогом от года создания ЦОПП']
        t_df.insert(0,'Наименование индикатора/показателя',temp.iloc[0,0])
        base_df = pd.concat([base_df,t_df],ignore_index=True,axis=0)
        # Создаем лист
    base_df.insert(0,'Регион',lst_regions)
    wb.create_sheet(title=str(column),index=idx)
    for row in dataframe_to_rows(base_df, index=False, header=False):
        wb[str(column)].append(row)

In [152]:
wb.save('Тест.xlsx')